In [7]:
from sympy import *
import networkx as nx
import numpy as np
from scipy import sparse
from scipy.sparse import linalg

In [33]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def get_num_spanning_trees(p, district):
    '''
    Given a partition p and a district number, this function returns the
    number of spanning trees in the subgraph of p corresponding to the 
    district. Uses Kirchoff's theorem to compute the number of spanning trees.
    
    Parameters:
        p is a partition
        district is the district number
    returns:
        the log() of the number of spanning trees
    '''
    graph = p.subgraphs[district]
    nodes = p.parts[district]
    
    #graph = nx.complete_graph(5)
    #nodes = [1,1,1,1,1]

    laplacian = nx.laplacian_matrix(graph)
    eigvals, eigvecs = linalg.eigs(laplacian.toarray())
    eigvals = np.absolute(eigvals)
    return np.absolute(1/len(nodes)*np.prod(eigvals, where = np.absolute(eigvals) > 0))

def get_num_spanning_trees2(p, district):
    '''
    Given a partition p and a district number, this function returns the log of the
    number of spanning trees in the subgraph of p corresponding to the 
    district. Uses Kirchoff's theorem to compute the number of spanning trees.
    
    Parameters:
        p is a partition
        district is the district number
    returns:
        the log() of the number of spanning trees
    '''
    graph = p.subgraphs[district]
    nodes = p.parts[district]
        
    #graph = nx.complete_graph(5)
    #nodes = [1,1,1,1,1]
    
    laplacian = nx.laplacian_matrix(graph)
    L = np.delete(np.delete(laplacian.todense(),0,0), 1,1)
    return exp(np.linalg.slogdet(L)[1])

## Testing!

In [2]:
# Generating an initial partition from Colorado shapefiles from here:
# https://github.com/mggg-states/CO-shapefiles

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas

graph = Graph.from_file("CO-shapefiles/co_precincts.shp")

elections = [
    Election("GOV18", {"Democratic": "GOV18D", "Republican": "GOV18R"})
]
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}
# Election updaters, for computing election results using the vote totals from our shapefile.
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)
initial_partition = GeographicPartition(graph, assignment="CD116FP", updaters=my_updaters)

In [38]:
# can set district_number to be anything '01' to '07'
district_number = '05'
print("number of spanning trees (by computing eigenvalues): ")
print(str(get_num_spanning_trees(initial_partition, district_number)))
print()
print("number of spanning trees (by computing determinant): ")
print(str(get_num_spanning_trees2(initial_partition, district_number)))

number of spanning trees (by computing eigenvalues): 
20364.71708558299

number of spanning trees (by computing determinant): 
4.80693066301099e+214
